In [7]:
from datasets import load_dataset

from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
aime24 = load_dataset("Maxwell-Jia/AIME_2024")

Generating train split: 30 examples [00:00, 3880.74 examples/s]


In [4]:
aime24

DatasetDict({
    train: Dataset({
        features: ['ID', 'Problem', 'Solution', 'Answer'],
        num_rows: 30
    })
})

In [6]:
aime24['train'][0]

{'ID': '2024-II-4',
 'Problem': 'Let $x,y$ and $z$ be positive real numbers that satisfy the following system of equations: \n\\[\\log_2\\left({x \\over yz}\\right) = {1 \\over 2}\\]\n\\[\\log_2\\left({y \\over xz}\\right) = {1 \\over 3}\\]\n\\[\\log_2\\left({z \\over xy}\\right) = {1 \\over 4}\\]\nThen the value of $\\left|\\log_2(x^4y^3z^2)\\right|$ is $\\tfrac{m}{n}$ where $m$ and $n$ are relatively prime positive integers. Find $m+n$.',
 'Solution': 'Denote $\\log_2(x) = a$, $\\log_2(y) = b$, and $\\log_2(z) = c$.\n\nThen, we have:\n$a-b-c = \\frac{1}{2}$,\n$-a+b-c = \\frac{1}{3}$,\n$-a-b+c = \\frac{1}{4}$.\n\nNow, we can solve to get $a = \\frac{-7}{24}, b = \\frac{-9}{24}, c = \\frac{-5}{12}$.\nPlugging these values in, we obtain $|4a + 3b + 2c|  = \\frac{25}{8} \\implies \\boxed{033}$.',
 'Answer': 33}

In [8]:
model_name = "Qwen/Qwen3-8B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

Loading checkpoint shards: 100%|████████████████████████████| 5/5 [00:11<00:00,  2.20s/it]


In [10]:
model.device

device(type='cuda', index=0)

In [11]:
model.generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "temperature": 0.6,
  "top_k": 20,
  "top_p": 0.95
}

In [25]:
prompt1 = "Please reason step by step, and put your final answer within \boxed{}. " 
prompt2 = "Answer the following question, and put your final answer within \boxed{}. "

prompt = prompt2 + aime24['train'][0]['Problem']

In [26]:
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [27]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

In [28]:
content = tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")
print("answer:", content)

answer: We are given the following system of logarithmic equations with positive real numbers $ x, y, z $:

$$
\log_2\left(\frac{x}{yz}\right) = \frac{1}{2} \tag{1}
$$
$$
\log_2\left(\frac{y}{xz}\right) = \frac{1}{3} \tag{2}
$$
$$
\log_2\left(\frac{z}{xy}\right) = \frac{1}{4} \tag{3}
$$

We are to find the value of:

$$
\left| \log_2(x^4 y^3 z^2) \right| = \left| 4\log_2 x + 3\log_2 y + 2\log_2 z \right|
$$

Let’s denote:

$$
a = \log_2 x, \quad b = \log_2 y, \quad c = \log_2 z
$$

Then the expressions become:

$$
\log_2\left(\frac{x}{yz}\right) = \log_2 x - \log_2(yz) = a - (b + c) = \frac{1}{2} \tag{4}
$$
$$
\log_2\left(\frac{y}{xz}\right) = \log_2 y - \log_2(xz) = b - (a + c) = \frac{1}{3} \tag{5}
$$
$$
\log_2\left(\frac{z}{xy}\right) = \log_2 z - \log_2(xy) = c - (a + b) = \frac{1}{4} \tag{6}
$$

So we now have the system of linear equations:

$$
\begin{cases}
a - b - c = \frac{1}{2} \tag{7} \\
b - a - c = \frac{1}{3} \tag{8} \\
c - a - b = \frac{1}{4} \tag{9}
\end{cases}
$$

Let’s